In [140]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [141]:
# В таблице user_actions сохраняются действия покупателей в интернет магазине, структура таблицы

# user_id    - id пользователя
# product_id - id товара
# action     - действие, просмотр товара, сохранение в корзину, покупка
# time       - время совершения события
# date       - дата события

# Для каждого дня рассчитайте, какой процент просмотров товаров завершился покупкой

In [224]:
all_user_ids = np.arange(1, 1001)
all_product_ids = np.arange(1, 101)

In [225]:
n = 10000

In [205]:
user_ids    = np.random.choice(all_user_ids, n)
product_ids = np.random.choice(all_product_ids, n)

In [208]:
start_date = pd.to_datetime('2022-01-01')

In [209]:
times = pd.date_range(start_date, periods=n, freq='1min')

In [211]:
user_actions = pd.DataFrame({'user_id': user_ids, 
                             'product_id': product_ids, 
                             'time': times})

In [212]:
user_actions['action'] = 'view'

In [213]:
user_actions.head()

,user_id,product_id,time,action
0,928,17,2022-01-01 00:00:00,view
1,839,10,2022-01-01 00:01:00,view
2,32,53,2022-01-01 00:02:00,view
3,832,68,2022-01-01 00:03:00,view
4,107,6,2022-01-01 00:04:00,view


In [214]:
def generate_funel_actions(user_id, product_id, time):
    to_cart = 0.2
    to_purchase = 0.4
    
    df = pd.DataFrame()
    
    if np.random.binomial(1, to_cart, 1)[0]:
        df = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(5, unit='s'), 
                           'action'    : 'add to cart'}, index=[0])
        
        if np.random.binomial(1, to_purchase, 1)[0]:
            df_purchase = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(10, unit='s'), 
                           'action'    : 'purchase'}, index=[0])
    
            df = df.append(df_purchase)
    return df

In [216]:
to_cart_df = pd.DataFrame()

In [217]:
for index, row in user_actions.iterrows():
    user_df = generate_funel_actions(row['user_id'], row['product_id'], row['time'])
    to_cart_df = to_cart_df.append(user_df)
    

In [219]:
user_actions = user_actions.append(to_cart_df)
user_actions = user_actions.sort_values('time')

In [220]:
user_actions['date'] = user_actions.time.dt.date

In [221]:
user_actions.head()

,user_id,product_id,time,action,date
0,928,17,2022-01-01 00:00:00,view,2022-01-01
1,839,10,2022-01-01 00:01:00,view,2022-01-01
2,32,53,2022-01-01 00:02:00,view,2022-01-01
3,832,68,2022-01-01 00:03:00,view,2022-01-01
4,107,6,2022-01-01 00:04:00,view,2022-01-01


In [201]:
q = """SELECT date, 
              views, 
              carts, 
              purchases, 
              100 * purchases / views as purchase_percantage 
       FROM (
           SELECT date, 
            count(case when action = 'view'        then 1 else NULL end) as views, 
            count(case when action = 'add to cart' then 1 else NULL end) as carts, 
            count(case when action = 'purchase'    then 1 else NULL end) as purchases 
           FROM user_actions 
           GROUP BY date);"""


In [223]:
sqldf(q)

,date,views,carts,purchases,purchase_percantage
0,2022-01-01,1440,287,133,9
1,2022-01-02,1440,287,127,8
2,2022-01-03,1440,265,105,7
3,2022-01-04,1440,303,133,9
4,2022-01-05,1440,285,105,7
5,2022-01-06,1440,262,111,7
6,2022-01-07,1360,248,106,7
